```
Let's use hashindex to solve Wordle and see if it's fast!

Here is the game hashindex played.
```

![wordle game](wordle.png)

In [1]:
# note: 469 us and 4.21ms were the timings of Frozen and Mutable respectively, prior to the rewrite.
# wonder how crappy they'll be now.

# hey nice!! Mutable is actually a little faster for whatever reason. 3.12ms now.
# And Frozen turned out faster too. 153 µs vs some 400 before. Very very cool.

In [1]:
import string
import time
from functools import partial
from hashindex import HashIndex, FrozenHashIndex
from words import all_wordle_words  # contains all 5-letter Wordle words as tuples of (word, frequency in English)

In [2]:
# Generate 5 functions to fetch the letter at each position in the word.
def char_at(pos, w):
    return w[0][pos]

def make_char_at_function(pos):
    p = partial(char_at, pos)
    p.__name__ = f'at_{pos}'
    return p

char_at = {pos: make_char_at_function(pos) for pos in range(5)}

In [3]:
# Generate 26 functions that count the occurrences of the specific letter.

def char_count(c, w):
    return w[0].count(c)

def make_char_count_function(letter):
    p = partial(char_count, letter)
    p.__name__ = f'count_{letter}'
    return p

char_count = {letter: make_char_count_function(letter) for letter in string.ascii_uppercase}

In [6]:
t0 = time.time()
hi = HashIndex(on=list(char_at.values()) + list(char_count.values()))
for w in all_wordle_words:
    hi.add(w)
t = time.time()-t0
print(t)
#  hi.freeze()

TypeError: getattr(): attribute name must be string

In [18]:
all_wordle_words[0]

('ABOUT', 1226734006)

In [6]:
len(all_wordle_words)*31

402132

In [19]:
ws = hi.find(
    match = {
    },
    exclude = {
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[('ABOUT', 1226734006),
 ('OTHER', 978481319),
 ('WHICH', 810514085),
 ('THEIR', 782849411),
 ('THERE', 701170205),
 ('FIRST', 578161543),
 ('WOULD', 572644147),
 ('THESE', 541003982),
 ('CLICK', 536746424),
 ('PRICE', 501651226)]

In [8]:
ws = hi.find(
    match = {
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[('STUDY', 152978354),
 ('UNTIL', 113090086),
 ('STUFF', 81444510),
 ('QUITE', 58777731),
 ('UNITS', 56512308),
 ('SUITE', 44829675),
 ('TRUTH', 35345925),
 ('TRUCK', 32136249),
 ('TRULY', 21693553),
 ('DUTCH', 19000979)]

In [9]:
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[('HUTCH', 1373107),
 ('CUTER', 152750),
 ('MUTCH', 106606),
 ('KUTCH', 67973),
 ('TUTEE', 17303),
 ('CUTCH', 6358),
 ('MUTER', 6358)]

In [10]:
%%timeit -n 25 -r 5 
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)

3.18 ms ± 127 µs per loop (mean ± std. dev. of 5 runs, 25 loops each)


```
Compare: find words by list comprehension
    
A frozen HashIndex is 10x faster or so. Not too shabby for such a small dataset!
```

In [11]:
%%timeit -n 25 -r 5 
z = [w for w in all_wordle_words if 
    w[0].count('S') == 0 and
    w[0].count('A') == 0 and
    w[0].count('B') == 0 and
    w[0].count('O') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('D') == 0 and
    w[0].count('N') == 0 and
    w[0].count('I') == 0 and
    w[0].count('L') == 0 and
    w[0][2] == 'T' and
    w[0].count('U') != 0 and
    w[0].count('T') != 0 and
    w[0][0] != 'U' and
    w[0][1] != 'T' and
    w[0][2] != 'U' and
    w[0][3] != 'U' and
    w[0][4] != 'T' 
]

3.98 ms ± 97.2 µs per loop (mean ± std. dev. of 5 runs, 25 loops each)


In [12]:
z = [w for w in all_wordle_words if 
    w[0].count('S') == 0 and
    w[0].count('A') == 0 and
    w[0].count('B') == 0 and
    w[0].count('O') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('D') == 0 and
    w[0].count('N') == 0 and
    w[0].count('I') == 0 and
    w[0].count('L') == 0 and
    w[0][2] == 'T' and
    w[0].count('U') != 0 and
    w[0].count('T') != 0 and
    w[0][0] != 'U' and
    w[0][1] != 'T' and
    w[0][2] != 'U' and
    w[0][3] != 'U' and
    w[0][4] != 'T' 
]
z

[('HUTCH', 1373107),
 ('CUTER', 152750),
 ('MUTCH', 106606),
 ('KUTCH', 67973),
 ('TUTEE', 17303),
 ('MUTER', 6358),
 ('CUTCH', 6358)]

In [13]:
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
ws

[('HUTCH', 1373107),
 ('CUTCH', 6358),
 ('TUTEE', 17303),
 ('CUTER', 152750),
 ('KUTCH', 67973),
 ('MUTCH', 106606),
 ('MUTER', 6358)]